# geoviews_mpl_map
---
Plots a simple ashfall model that can be projected into an arbitrary coordinate system. Drag the slider to view the ashfall map at different time steps. A very solid, basic evaluative tool.

## *Pros*
* Very minimal code required to produce a very nice product
* Plays well with cartopy &mdash; that means features, projections etc.
* Very snappy interface
* Matplotlib backend makes contours etc. much easier to implement

## *Cons*
* No Bokeh integration means no panning / zooming functionality and no HTML export
* Time slider is not very configurable

In [ ]:
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import cartopy.feature as cf
import colorcet as cc

import sys
sys.path.insert(0, '../')
from vis_tools import read_hysplit_netcdf

# ignore two benign warnings that arise from my use of NaN's instead of 0's
import warnings
warnings.filterwarnings('ignore', message='invalid value encountered in less_equal')
warnings.filterwarnings('ignore', message='All-NaN slice encountered')

hv.extension('matplotlib')

###############################################
# SPECIFY: file name and path for HYSPLIT model
FILENAME = '../18042918_taupo_15.0_0.01.nc'

# SPECIFY:
ASH_MIN = 10**-1  # min ash colorbar cutoff
ASH_MAX = 10**2  # max ash colorbar cutoff
###############################################

NZTM_PROJ = ccrs.epsg(2193)  # define projection using NZTM EPSG code (need internet access)

# INPUT: boundary data
boundary_data = cf.GSHHSFeature(scale='auto', levels=[1, 2]) 
      
# INPUT: model   
model = read_hysplit_netcdf(FILENAME, ASH_MIN)

gv_ds = gv.Dataset(model, crs=ccrs.PlateCarree())
gv_ds = gv_ds.redim.label(lon='easting', lat='northing', total_deposition='ash thickness (mm)')

fig = gv_ds.to(gv.Image, ['lon', 'lat']) * gv.Feature(boundary_data) * gv.Points(tuple(model.attrs['volcano_location'][::-1]))

plot_opts = {'Image': {'style': dict(cmap=cc.m_fire_r, clims=(ASH_MIN, ASH_MAX)),
                        'plot': dict(projection=NZTM_PROJ, colorbar=True, logz=True, fig_size=200,
                                     title_format=FILENAME.split('/')[-1])},
           'Feature': {'style': dict(),
                        'plot': dict(projection=NZTM_PROJ)},
            'Points': {'style': dict(marker='^', color='c', edgecolors='k', s=100, linewidths=0.75),
                        'plot': dict(projection=NZTM_PROJ)}            
            }

fig = fig.opts(plot_opts)

fig